In [1]:
#import necessary package and collect initial variables
import openmdao.api as om
%run ../initialvars.ipynb

In [2]:
#define discipline group and call analyses 
class stab(om.Group):
    def setup(self):
    

IndentationError: expected an indented block (266218981.py, line 4)